In [1]:
import os
import shutil 
import csv
import numpy as np
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import StandardScaler
from utils import *

Using TensorFlow backend.


In [7]:
def get_labels_y1(data):
    data_new = []
    print(len(data))
    for i in range(len(data)):
        data_new.append(data[i] + [1,1,1,1,1,1])
        
    return data_new

In [2]:
band_name = 'v3'  
window_size = 6 

In [3]:
path = 'sources/Ini_sources/'
#print(os.getcwd())
data_1 = make_sequences_y1(band_name, path, band_name)
print(len(data_1))
print(data_1)

16
[[2.4605200000000877, 2.4666939999999613, 2.4781299999999646, 2.4949169999999867, 2.5168509999998605, 2.5409810000001016, 2.5749949999999444, 2.610016000000087, 2.6477270000000317, 2.6896099999999024, 2.734294000000091, 2.7827760000000126, 2.834677000000056, 2.8897729999998774, 2.9384440000001177, 3.00591600000007, 3.073649999999816, 3.143542000000025, 3.216996999999992, 3.2950650000000223, 3.385539000000108], [2.497233999999935, 2.505493000000115, 2.5208950000001096, 2.5371829999999136, 2.5582950000000437, 2.5884540000001834, 2.620439999999917, 2.655993999999964, 2.695167999999967, 2.7391020000000026, 2.786493999999948, 2.8373350000001665, 2.8943719999999757], [2.500084999999899, 2.5129750000000968, 2.5340899999998783, 2.5633940000000166, 2.595263000000159, 2.6357949999999164, 2.68797399999994, 2.7343960000000607, 2.7921759999999267, 2.8572240000000875, 2.9255269999998745, 2.99732999999992, 3.0770649999999478, 3.1554849999999988, 3.2395950000000084, 3.3270410000000084, 3.4208150000

In [287]:
''' Writing out initial sequences
file_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')
for line in data_1:
    file_ini.write(str(line) +'\n')
    
file_ini.close()

'''

" Writing out initial sequences\nfile_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')\nfor line in data_1:\n    file_ini.write(str(line) +'\n')\n    \nfile_ini.close()\n\n"

In [4]:
from decimal import Decimal, ROUND_HALF_UP, getcontext

if band_name == 'v3':
    getcontext().rounding = ROUND_HALF_UP
    l = len(data_1)
    h = int(round(Decimal(l)/2,0))
    i = int(round(Decimal(h)/2,0))
    indexes = [i, l-(h-i)]
    print(indexes)
    
    train = data_1[:indexes[0]] + data_1[indexes[1]:]
    test  = data_1[indexes[0]:indexes[1]]
    print(len(train))

[4, 12]
8


In [8]:
if band_name != 'v3':
    test = data_1
data_test_1 = get_x(test, window_size=window_size).tolist()
data_out = get_labels_y1(data_test_1)

data_out = np.array(data_out)

file_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y1','Y2','Y3','Y4','Y5','Y6'])
for i in range(data_out.shape[0]):
    filewriter.writerow(data_out[i,:])
file_csv.close()


103


# Creating additional samples for train set

In [9]:
from statistics import mean 

def check_data_uniform_chaning(data, out=False):
    data_to_continue = []
    for line in data:
        flag = True
        temp = line[1] - line[0]
        if temp > 0:
            for i in range(1, len(line[:-1])):
                if line[i+1] < line[i]:
                    flag = False
                    break
        else:
            for i in range(1,len(line[:-1])):
                if line[i+1] > line[i]:
                    flag = False
                    break
        if flag is False:
            if out is True: print(line)
            new = []
            #m = mean(line)
            #for i in line:
            #    new.append(i-m)
            #print(new)
        else:
            data_to_continue.append(line)
    return data_to_continue


In [10]:
def negative_sequences(data):
    data_new = []
    for el in data:
        seq = []
        for cell in el:
            seq.append(-cell)
        data_new.append(seq)

    return data_new

In [11]:
def reverse_sequences(data, out=False):
    data_new = []
    for line in data:
        line_new = line[::-1]
        if out is True:
            print('ini:\t', line)
            print('reversed:\t', line_new)
        data_new.append(line_new)
    return data_new

In [12]:
def calculate_deltas(line):
    deltas = [0]
    for i in range(len(line)-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

def create_shift_in_deltas(data, shift, out=False):
    data_add = []

    for line in data:
        deltas = calculate_deltas(line)
        min_delta = min(deltas[1:])
        if abs(shift) > abs(min_delta):
            continue
        new = [line[0]]
        ini = line[0]
        for i in range(1, len(line)):
            ini = ini - deltas[i] - shift
            new.append(ini)
        if out is True:
            print('ini:\t', line)
            print('ini deltas:\t', deltas)
            print('new:\t', new)
            #d = calculate_deltas(new)
            #print(d)
            #d2 = calculate_deltas(d[1:])
            #print(d2)
            #print(min(d2))
        data_add.append(new)
    
    return data_add

In [13]:
file_csv = open('sources/train_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y1','Y2','Y3','Y4','Y5','Y6'])


31

In [14]:
data = get_x(train, window_size=window_size).tolist()
print('ini:\t\t', len(data))

data_add_negative = negative_sequences(data)
print('negative:\t', len(data_add_negative))

data += data_add_negative
print('total:\t\t', len(data))

data_out = get_labels_y1(data)

data_out = np.array(data_out)
for i in range(data_out.shape[0]):
    filewriter.writerow(data_out[i,:])


ini:		 91
negative:	 91
total:		 182
182


In [15]:
data = check_data_uniform_chaning(data, out=False)
print('only uniform changing in sequence:\t', len(data))   

only uniform changing in sequence:	 182


In [16]:
data_add_reversed = reverse_sequences(data, out=False)
print('ini:\t\t', len(data))
print('reversed:\t', len(data_add_reversed))

data += data_add_reversed
print('total:\t\t', len(data))

ini:		 182
reversed:	 182
total:		 364


In [17]:
print('ini:\t\t', len(data))
shifts = [i for i in np.arange(-0.015, 0.015, 0.0005) if abs(i) > 0.00001]
data_add_res = []
for shift in shifts:
    data_add = create_shift_in_deltas(data, shift, out=False)
    data_add_res += data_add
print('shifted:\t', len(data_add_res))

data += data_add_res
print('total:\t\t', len(data))

ini:		 364
shifted:	 21304
total:		 21668


In [18]:
data_out = get_labels_y1(data)

data_out = np.array(data_out)
for i in range(data_out.shape[0]):
    filewriter.writerow(data_out[i,:])
    

21668


In [19]:
file_csv.close()

In [ ]:
'''Muliplicate deltas by 10s
data_add = []
for line in data:
    deltas = create_deltas(line, y_label=True)
    #print(line)
    #print('d ', deltas)
    if all(delta <= 0.1 for delta in deltas) is True:
        cell_max = m
        ax(map(abs,deltas))
        cell_min = min(map(abs,deltas[1:]))
        while (cell_max < 0.05):
            deltas = [d*10 for d in deltas]
            cell_max *= 10
            new = []
            ini = line[0]
            for i in range(len(deltas)):
                ini = ini - deltas[i]
                new.append(ini)
            new.append(1.)
            if abs(line[0] - 3.5738) <= 0.0001:
                print(line)
                print(create_deltas(line, y_label=True))
                print('1', new)
    
            #print('n ',new)
            data_add.append(new)
            
            if cell_min >= 0.002:
                deltas2 = [d/2 for d in deltas]
                new = []
                #print('1', deltas)
                #print('2',deltas2)
                ini = line[0]
                for i in range(len(deltas2)):
                    ini = ini - deltas2[i]
                    new.append(ini)
                new.append(1.)
                if abs(line[0] - 3.5738) <= 0.0001:
                    print('2',new)
                #print('n ',new)
                data_add.append(new)
            
print(len(data_add))
'''